Experimentations with Stanza

---

## Imports

In [143]:
!pip install gradio
import gradio as gr
import random
import stanza
from gradio.mix import Series
import string

---
## Read in Data

In [144]:
# define the file name here:
quiz = 'lib/quiz_questions.txt'

# list to hold each line of the file
lines = []
# list of dictionaries for holding the English answer/ Greek answer
exercises = []

# Read in the lines from the file
with open(quiz) as f:
    # create list for holding the exercises
    lines = f.readlines()

# For each line, use regex to grab the answer and full sentence
for sent in lines:
    
    # Get the greek answer
    eng_ans_end = sent.find(':')
    english_answer = sent[0:eng_ans_end]

    greek_answer = sent[eng_ans_end+1:]
    
    # Add everything to our list of dictionaries
    exercises.append({"english answer":english_answer, "greek answer":greek_answer})
print(exercises)

[{'english answer': '[He] walked  in silence to [the] shore of [the] loudly-roaring sea.', 'greek answer': " βῆ δ' ἀκέων παρὰ θῖνα πολυφλοίσβοιο θαλάσσης.\n"}, {'english answer': 'And [she] became frightened, [the] ox-eyed, revered Hera', 'greek answer': ' ἔδεισεν δὲ βοῶπις πότνια Ἥρη\n'}, {'english answer': 'Sure enough,  [you] said all these things  rightly, old man.', 'greek answer': ' ναὶ δὴ ταῦτά γε πάντα γέρον κατὰ μοῖραν ἔειπες·'}]


In [145]:
# randomly choose one of the sentences
sentence = random.choice(exercises)["greek answer"]
# get rid of punctuation
sentence = sentence.translate(str.maketrans('', '', string.punctuation))
print(sentence)


 βῆ δ ἀκέων παρὰ θῖνα πολυφλοίσβοιο θαλάσσης



## Use Stanza for Analysis

In [146]:
# stanza.download('grc') 
nlp = stanza.Pipeline('grc') 
doc = nlp(sentence) 

print(doc)
print(doc.entities)

2022-07-06 15:29:19 INFO: Loading these models for language: grc (Ancient_Greek):
| Processor | Package |
-----------------------
| tokenize  | proiel  |
| pos       | proiel  |
| lemma     | proiel  |
| depparse  | proiel  |

2022-07-06 15:29:19 INFO: Use device: cpu
2022-07-06 15:29:19 INFO: Loading: tokenize
2022-07-06 15:29:19 INFO: Loading: pos
2022-07-06 15:29:19 INFO: Loading: lemma
2022-07-06 15:29:19 INFO: Loading: depparse
2022-07-06 15:29:20 INFO: Done loading processors!


[
  [
    {
      "id": 1,
      "text": "βῆ",
      "lemma": "βή",
      "upos": "NOUN",
      "xpos": "Nb",
      "feats": "Case=Nom|Gender=Neut|Number=Sing",
      "head": 0,
      "deprel": "root",
      "start_char": 1,
      "end_char": 3
    },
    {
      "id": 2,
      "text": "δ",
      "lemma": "δ",
      "upos": "NUM",
      "xpos": "Ma",
      "head": 3,
      "deprel": "nummod",
      "start_char": 4,
      "end_char": 5
    },
    {
      "id": 3,
      "text": "ἀκέων",
      "lemma": "ἄκον",
      "upos": "NOUN",
      "xpos": "Nb",
      "feats": "Case=Gen|Gender=Neut|Number=Plur",
      "head": 1,
      "deprel": "nmod",
      "start_char": 6,
      "end_char": 11
    },
    {
      "id": 4,
      "text": "παρὰ",
      "lemma": "παρά",
      "upos": "ADP",
      "xpos": "R-",
      "head": 5,
      "deprel": "case",
      "start_char": 12,
      "end_char": 16
    },
    {
      "id": 5,
      "text": "θῖνα",
      "lemma": "θῖνος",
      "upos": "NOUN",
      "xpos":

Declare the part of speech here:

In [147]:
part_of_speech = "NOUN"

Find the correct answers

In [148]:
all_words = [word.text for sent in doc.sentences for word in sent.words]
print(all_words)

correct_answers = [word.text for sent in doc.sentences for word in sent.words if word.upos==part_of_speech]
print(correct_answers)

['βῆ', 'δ', 'ἀκέων', 'παρὰ', 'θῖνα', 'πολυφλοίσβοιο', 'θαλάσσης']
['βῆ', 'ἀκέων', 'θῖνα', 'θαλάσσης']
['δ', 'παρὰ', 'πολυφλοίσβοιο']


## User Interface

In [155]:
def check(answer):
    if not len(answer) == len(correct_answers):
        return "Try selecting more or fewer answers"
    for a in answer:
        if a not in correct_answers:
            return "Not Quite"
    return "Correct!"


sent = "Identify all of the " + str(part_of_speech) + "(s) in the following sentence:"

demo = gr.Interface(description=sent, fn=check, inputs=[gr.CheckboxGroup(all_words, label=sentence)], outputs="text")

print(correct_answers)
demo.launch()

['βῆ', 'ἀκέων', 'θῖνα', 'θαλάσσης']
Running on local URL:  http://127.0.0.1:7907/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7febd789ba60>, 'http://127.0.0.1:7907/', None)

Exception in callback None(<Task finishe...> result=None>)
handle: <Handle>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/asyncio/events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
TypeError: 'NoneType' object is not callable
